1. RNN

    1.1 RNNを設計する主な原因

    シーケンスデータの処理
    動機: 自然言語処理（NLP）、音声認識、時系列予測など、入力データが時間的な順序を持つタスクに対応するため。RNNは、これらのシーケンス内のパターンや依存関係を学習する能力があります。
    
    時間的な依存関係の把握
    動機: 過去のデータが現在の出力に重要な影響を与える場合、RNNはそのような時間的な依存関係をモデル化できます。例えば、言語モデルでは、文の意味を理解するために前の単語の文脈が必要です。
    
    可変長の入力・出力
    動機: 固定サイズの入力を扱う従来のニューラルネットワークと異なり、RNNは可変長の入力シーケンスを処理することができます。これにより、任意の長さのテキストや音声データを扱うことが可能になります。

    1.2 考慮事項

    長期依存性の問題
    
    課題: 標準的なRNNは、長期依存性の問題（勾配消失・勾配爆発）により、長いシーケンスデータでの学習が困難です。この問題を克服するために、LSTM（Long Short-Term Memory）やGRU（Gated Recurrent Unit）などの改良されたRNNが提案されています。

    パラメータのチューニング
    課題: RNNの学習率や隠れ層のユニット数などのハイパーパラメータの選択が、モデルの性能に大きく影響します。適切なハイパーパラメータを見つけるためには、多くの実験が必要になることがあります。
    
    計算コスト
    課題: RNNはシーケンスの各時点で前の時点の隠れ状態を利用するため、並列計算が難しく、特に長いシーケンスを扱う場合には計算コストが高くなりがちです。
    
    過学習の防止
    課題: RNNも他の深層学習モデルと同様に過学習（トレーニングデータに過剰に適合すること）のリスクがあります。これを防ぐために、ドロップアウトや正則

2. LSTMとGRU

    2.1 単純なRNNの欠点

    単純なRNN（基本的な再帰的ニューラルネットワーク）は、理論上は時系列データの特徴を学習する強力な能力を持っていますが、実際には「長期依存性の問題」に直面します。この問題は、シーケンスが長くなるにつれて、初期の情報がネットワークを通過する際に失われることを意味します。具体的には、勾配消失や勾配爆発の問題が生じ、モデルが長期間の依存関係を効果的に学習することが難しくなります。

    2.2 LSTM

    LSTMは、入力ゲート、忘却ゲート、出力ゲートの3つの主要なゲートを持っています。これらのゲートは、それぞれ新しい情報を隠れ状態にどの程度追加するか、隠れ状態から情報をどの程度忘れるか、隠れ状態から次の層へとどの程度の情報を出力するかを制御します。LSTMは、多くの自然言語処理や音声認識タスクにおいて、高い性能を発揮します。

    ![LSTM](./image/LSTM.png)

    input gateは、セル状態を更新する役割を果たします。
    
    forget gateでは、通過させる情報と通化させない情報を決定します。
    
    output gateは、隠れユニットの値の更新方法を決定します。

    ![LSTM_structure](./image/LSTM_structure.png)

    ![LSTM_calculation](./image/LSTM_calculation.png)


    2.3 GRU

    GRUはLSTMを簡略化したバージョンであり、リセットゲートと更新ゲートの2つのゲートを使用します。これらは、新しい情報の統合方法と、過去の情報をどの程度保持するかを制御します。GRUはLSTMに比べてパラメータが少ないため、計算効率が良く、小さなデータセットでの学習が速くなる傾向があります。

    ![GRU](./image/GRU.png)
    ![GRU](./image/GRU_calculation.png)




# simpleRNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# RNNモデルの定義
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        # RNNレイヤーの定義
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        # 出力層の定義
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        # 初期隠れ状態をゼロに設定
        h0 = torch.zeros(1, x.size(0), self.hidden_size)
        
        # RNN層の出力と隠れ状態を計算
        out, hn = self.rnn(x, h0)
        
        # 最後のタイムステップの隠れ状態を出力層に通す
        out = self.fc(out[:, -1, :])
        return out

# ハイパーパラメータの設定
input_size = 1  # 入力特徴量の数
hidden_size = 10  # 隠れ層のサイズ
output_size = 1  # 出力特徴量の数
learning_rate = 0.01
num_epochs = 100

# モデル、損失関数、最適化手法の定義
model = SimpleRNN(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# トレーニングデータの準備（例として単純な数列）
# 入力データのサイズは (バッチサイズ, シーケンス長, 特徴量の数)
x_train = torch.tensor([
    [[0.1], [0.2], [0.3], [0.4], [0.5]],
    [[0.2], [0.3], [0.4], [0.5], [0.6]],
    [[0.3], [0.4], [0.5], [0.6], [0.7]],
    [[0.4], [0.5], [0.6], [0.7], [0.8]]
], dtype=torch.float32)

# 目標値
y_train = torch.tensor([[0.6], [0.7], [0.8], [0.9]], dtype=torch.float32)

# トレーニングループ
for epoch in range(num_epochs):
    # 順伝播
    outputs = model(x_train)
    loss = criterion(outputs, y_train)
    
    # 逆伝播と最適化
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# テスト
model.eval()  # 評価モードに切り替え
with torch.no_grad():
    test_input = torch.tensor([[[0.5], [0.6], [0.7], [0.8], [0.9]]], dtype=torch.float32)
    test_output = model(test_input)
    print(f'Test Input: {test_input.tolist()}')
    print(f'Predicted Output: {test_output.item():.4f}')


# LSTM

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# LSTMモデルの定義
class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), hidden_size)  # 初期隠れ状態
        c0 = torch.zeros(1, x.size(0), hidden_size)  # 初期セル状態
        out, _ = self.lstm(x, (h0, c0))  # LSTM層の順伝播
        out = self.fc(out[:, -1, :])  # 最後のタイムステップの出力
        return out

# モデルのインスタンス化とハイパーパラメータ設定
input_size = 1
hidden_size = 10
output_size = 1
learning_rate = 0.01
num_epochs = 100

model = SimpleLSTM(input_size, hidden_size, output_size)
criterion = nn.MSELoss()  # 損失関数
optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # 最適化手法

# トレーニングデータの準備（例として単純な数列）
x_train = torch.tensor([
    [[0.1], [0.2], [0.3], [0.4], [0.5]],
    [[0.2], [0.3], [0.4], [0.5], [0.6]],
    [[0.3], [0.4], [0.5], [0.6], [0.7]],
    [[0.4], [0.5], [0.6], [0.7], [0.8]]
], dtype=torch.float32)
y_train = torch.tensor([[0.6], [0.7], [0.8], [0.9]], dtype=torch.float32)

# モデルのトレーニング
for epoch in range(num_epochs):
    outputs = model(x_train)
    loss = criterion(outputs, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# テスト
model.eval()
with torch.no_grad():
    test_input = torch.tensor([[[0.5], [0.6], [0.7], [0.8], [0.9]]], dtype=torch.float32)
    test_output = model(test_input)
    print(f'Test Input: {test_input.tolist()}')
    print(f'Predicted Output: {test_output.item():.4f}')


In [ ]:
# GRU

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# GRUモデルの定義
class SimpleGRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleGRU, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), hidden_size)  # 初期隠れ状態
        out, _ = self.gru(x, h0)  # GRU層の順伝播
        out = self.fc(out[:, -1, :])  # 最後のタイムステップの出力
        return out

# モデルのインスタンス化とハイパーパラメータ設定
input_size = 1
hidden_size = 10
output_size = 1
learning_rate = 0.01
num_epochs = 100

model = SimpleGRU(input_size, hidden_size, output_size)
criterion = nn.MSELoss()  # 損失関数
optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # 最適化手法

# トレーニングデータの準備（例として単純な数列）
x_train = torch.tensor([
    [[0.1], [0.2], [0.3], [0.4], [0.5]],
    [[0.2], [0.3], [0.4], [0.5], [0.6]],
    [[0.3], [0.4], [0.5], [0.6], [0.7]],
    [[0.4], [0.5], [0.6], [0.7], [0.8]]
], dtype=torch.float32)
y_train = torch.tensor([[0.6], [0.7], [0.8], [0.9]], dtype=torch.float32)

# モデルのトレーニング
for epoch in range(num_epochs):
    outputs = model(x_train)
    loss = criterion(outputs, y_train)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# テスト
model.eval()
with torch.no_grad():
    test_input = torch.tensor([[[0.5], [0.6], [0.7], [0.8], [0.9]]], dtype=torch.float32)
    test_output = model(test_input)
    print(f'Test Input: {test_input.tolist()}')
    print(f'Predicted Output: {test_output.item():.4f}')
